### 데이터 셋 

In [1]:
import pandas as pd
import numpy as np

# 센서 데이터 (1시간 동안 5분 간격)
sensor = pd.DataFrame({
    'timestamp': pd.date_range('2024-01-01 09:00', periods=12, freq='5min'),
    'machine_id': ['M001']*6 + ['M002']*6,
    'temperature': [35.0, 36.2, 38.5, 40.1, 39.8, 37.5,  # M001
                    42.0, 43.5, 45.0, 44.2, 43.0, 41.5]  # M002
})

print(sensor)

             timestamp machine_id  temperature
0  2024-01-01 09:00:00       M001         35.0
1  2024-01-01 09:05:00       M001         36.2
2  2024-01-01 09:10:00       M001         38.5
3  2024-01-01 09:15:00       M001         40.1
4  2024-01-01 09:20:00       M001         39.8
5  2024-01-01 09:25:00       M001         37.5
6  2024-01-01 09:30:00       M002         42.0
7  2024-01-01 09:35:00       M002         43.5
8  2024-01-01 09:40:00       M002         45.0
9  2024-01-01 09:45:00       M002         44.2
10 2024-01-01 09:50:00       M002         43.0
11 2024-01-01 09:55:00       M002         41.5


### 데이터 타입, 기본분석

In [2]:
print(sensor.info())
print()
print(sensor.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    12 non-null     datetime64[ns]
 1   machine_id   12 non-null     object        
 2   temperature  12 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 420.0+ bytes
None

                           timestamp  temperature
count                             12    12.000000
mean   2024-01-01 09:27:29.999999744    40.525000
min              2024-01-01 09:00:00    35.000000
25%              2024-01-01 09:13:45    38.250000
50%              2024-01-01 09:27:30    40.800000
75%              2024-01-01 09:41:15    43.125000
max              2024-01-01 09:55:00    45.000000
std                              NaN     3.230853


In [3]:
# mechine_id의 조회, 이용 성능 항샹을 위해 데이터형 바꾼다.
sensor['machine_id']= sensor['machine_id'].astype('string') # 결측치가 없어서 해당으로 적용가능  🐚 'String'으로 적어야함.
# 🐚 to_str는 없는거같던데? -> 없음.

print(sensor.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    12 non-null     datetime64[ns]
 1   machine_id   12 non-null     string        
 2   temperature  12 non-null     float64       
dtypes: datetime64[ns](1), float64(1), string(1)
memory usage: 420.0 bytes
None


### 전체 15분 단위 평균 온도
- 시간 단위의 평균 온도를 구한다. 시간을 인덱스로 하고 resample('15m')을 통해 평균 온도를 구한다. 새로운 테이블 or 붙이기로 각각


In [4]:
sensor = sensor.set_index('timestamp')

In [5]:
sensor['15min_avr'] = sensor.resample('15T')['temperature'].mean() # 공백 존재함. 싫으면 sensor.resample('15T')['temperature'].trasnform('mean') 🐚 분은 min or T 주의할것. 
print(sensor)
print()
new_sensor = sensor.resample('15T')['temperature'].mean() # 15분 평균만 존재하는 테이블 생성 
print(new_sensor)

                    machine_id  temperature  15min_avr
timestamp                                             
2024-01-01 09:00:00       M001         35.0  36.566667
2024-01-01 09:05:00       M001         36.2        NaN
2024-01-01 09:10:00       M001         38.5        NaN
2024-01-01 09:15:00       M001         40.1  39.133333
2024-01-01 09:20:00       M001         39.8        NaN
2024-01-01 09:25:00       M001         37.5        NaN
2024-01-01 09:30:00       M002         42.0  43.500000
2024-01-01 09:35:00       M002         43.5        NaN
2024-01-01 09:40:00       M002         45.0        NaN
2024-01-01 09:45:00       M002         44.2  42.900000
2024-01-01 09:50:00       M002         43.0        NaN
2024-01-01 09:55:00       M002         41.5        NaN

timestamp
2024-01-01 09:00:00    36.566667
2024-01-01 09:15:00    39.133333
2024-01-01 09:30:00    43.500000
2024-01-01 09:45:00    42.900000
Freq: 15min, Name: temperature, dtype: float64


/var/folders/bf/_c9d0b9571x410m1fmyxr4yh0000gn/T/ipykernel_48072/3718659051.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  sensor['15min_avr'] = sensor.resample('15T')['temperature'].mean() # 공백 존재함. 싫으면 sensor.resample('15T')['temperature'].trasnform('mean') 🐚 분은 min or T 주의할것.
/var/folders/bf/_c9d0b9571x410m1fmyxr4yh0000gn/T/ipykernel_48072/3718659051.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  new_sensor = sensor.resample('15T')['temperature'].mean() # 15분 평균만 존재하는 테이블 생성


### 3개 이동 평균 컬럼 추가
- 개수로 이동평균 이니까 rolling(3).mean()두면 될듯. 최소정한거도 없고 피쳐를 새롭게 생성하면 됨. 

In [6]:
sensor['3AVG_Moving'] = sensor.rolling(3)['temperature'].mean()
sensor

,machine_id,temperature,15min_avr,3AVG_Moving
timestamp,,,,
2024-01-01 09:00:00,M001,35.0,36.566667,NaN
2024-01-01 09:05:00,M001,36.2,NaN,NaN
2024-01-01 09:10:00,M001,38.5,NaN,36.566667
2024-01-01 09:15:00,M001,40.1,39.133333,38.266667
2024-01-01 09:20:00,M001,39.8,NaN,39.466667
2024-01-01 09:25:00,M001,37.5,NaN,39.133333
2024-01-01 09:30:00,M002,42.0,43.500000,39.766667
2024-01-01 09:35:00,M002,43.5,NaN,41.000000
2024-01-01 09:40:00,M002,45.0,NaN,43.500000


### 기계별 + 15분 단위 temperature 평균

- groupby로 machine_id,15분단위를 묶어서 나타낸다 

In [7]:
# sensor['기계별_15분온도평균']= sensor.groupby('machine_id',as_index=False).resample('15T')['temperature'].transform('mean') #🐚 가능하긴했으나 다른방법도 존재한다. 
sensor = sensor.reset_index()
sensor['기계별_15분온도평균']=sensor.groupby(['machine_id',pd.Grouper(key='timestamp',freq='15min')])['temperature'].transform('mean')
sensor

,timestamp,machine_id,temperature,15min_avr,3AVG_Moving,기계별_15분온도평균
0,2024-01-01 09:00:00,M001,35.0,36.566667,NaN,36.566667
1,2024-01-01 09:05:00,M001,36.2,NaN,NaN,36.566667
2,2024-01-01 09:10:00,M001,38.5,NaN,36.566667,36.566667
3,2024-01-01 09:15:00,M001,40.1,39.133333,38.266667,39.133333
4,2024-01-01 09:20:00,M001,39.8,NaN,39.466667,39.133333
5,2024-01-01 09:25:00,M001,37.5,NaN,39.133333,39.133333
6,2024-01-01 09:30:00,M002,42.0,43.500000,39.766667,43.500000
7,2024-01-01 09:35:00,M002,43.5,NaN,41.000000,43.500000
8,2024-01-01 09:40:00,M002,45.0,NaN,43.500000,43.500000
9,2024-01-01 09:45:00,M002,44.2,42.900000,44.233333,42.900000


### 직전 대비 2도 이상 상승 시점

- 시간의 흐름에 따라 가느넫 이전보다 2도이상이면 말하는거면 좋겠는데 뭐가 있을라나 흠 아니면 시간의 흐름으로 안가더라도 이전값이랑 비교하는 함수는? 
- 🐚 diff()함수 = 현재값 - 이전값 (자동계산)
- 🐚 diff 형태 : df['피처명'].diff() 자동으로 함 

In [ ]:
# 직전 대비 2도 이상 이면 정렬을 필수
sensor = sensor.sort_values(['machine_id','timestamp'],ascending=[True,True]) # 🐚정렬 순서 중요 

# 이전과 비교해서 2도 이상인거 체크  🐚 첫 행은 비교할게 없어서 NaN라 fillna(0)해주기
sensor['이전온도비교']= sensor['temperature'].diff().fillna(0) 
# 🐚 위 코드의 치명적인 단점 : id가 달라졌는데 온도변화는 있다고 함. 따라서 id별로 계산을 진행해야한다. -> ⭐️ 이게 맞는코드 ⭐️ sensor['이전온도비교']= sensor.groupby('machine_id')['temperature'].diff().fillna(0)

# 2도 이상 차이나는구간 체크
# np.select(조건변수,결과변수,default)
condition = [sensor['이전온도비교'] >= 2,sensor['이전온도비교'] <= -2]
result = ['2도이상상승','2도이하하락']
sensor['2도이상상승구간'] = np.select(condition,result,default='정상')
sensor

,timestamp,machine_id,temperature,15min_avr,3AVG_Moving,기계별_15분온도평균,이전온도비교,2도이상상승구간
0,2024-01-01 09:00:00,M001,35.0,36.566667,NaN,36.566667,0.0,정상
1,2024-01-01 09:05:00,M001,36.2,NaN,NaN,36.566667,1.2,정상
2,2024-01-01 09:10:00,M001,38.5,NaN,36.566667,36.566667,2.3,2도이상상승
3,2024-01-01 09:15:00,M001,40.1,39.133333,38.266667,39.133333,1.6,정상
4,2024-01-01 09:20:00,M001,39.8,NaN,39.466667,39.133333,-0.3,정상
5,2024-01-01 09:25:00,M001,37.5,NaN,39.133333,39.133333,-2.3,2도이하하락
6,2024-01-01 09:30:00,M002,42.0,43.500000,39.766667,43.500000,4.5,2도이상상승
7,2024-01-01 09:35:00,M002,43.5,NaN,41.000000,43.500000,1.5,정상
8,2024-01-01 09:40:00,M002,45.0,NaN,43.500000,43.500000,1.5,정상
9,2024-01-01 09:45:00,M002,44.2,42.900000,44.233333,42.900000,-0.8,정상


### 기계별 최고 온도 시점

In [ ]:
# 1. 기계별로 그룹화하되 temperature 기준 max 묶는다 이거를 하나의 피처로 만든다
# 2. max_temp와 temp가 일치할때의 timestamp를 출력한다

sensor['max_temp'] = sensor.groupby('machine_id')['temperature'].transform('max')
sensor[sensor['max_temp'] == sensor['temperature']][['timestamp','machine_id','max_temp']]


,timestamp,machine_id,max_temp
3,2024-01-01 09:15:00,M001,40.1
8,2024-01-01 09:40:00,M002,45.0
